In [1]:
import cv2
import numpy as np
import os
import math
import json

In [2]:
# Contour parameters
min_contour_width = 30  
min_contour_height = 30
max_contour_width = 100 
max_contour_height = 100
# Distance from the line
offset = 15
# Lane parameters
lane_3_height_1=150
lane_3_height_2=160
lane_2_height_1=180
lane_2_height_2=190
lane_3_width_1=280
lane_3_width_2=260
lane_2_width_1=230
lane_2_width_2=210
matches = []
# Vehicle counters
vehicles_3 = 0
vehicles_2 = 0
# Other helping metrics
frame_number_threshold=16
new_x_2=0
new_y_2=0
new_x_3=0
new_y_3=0
last_2=""
last_3=""

In [3]:
# Line to point distance function

def point_to_line_distance(point, line_start_width, line_start_height, line_end_width, line_end_height):

    A = line_end_height - line_start_height
    B = line_start_width - line_end_width
    C = line_end_width * line_start_height - line_start_width * line_end_height

    distance = abs(A * point[0] + B * point[1] + C) / np.sqrt(A**2 + B**2)
    return distance

In [4]:
# Point to point distance function
def distance_between_points(x1, y1, x2, y2):

    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return distance

In [5]:
# Getting centroid
def get_centrolid(x, y, w, h):
   x1 = int(w / 2)
   y1 = int(h / 2)
 
   cx = x + x1
   cy = y + y1
   return cx, cy

In [6]:
# Loading videos
files=[]
folder_path = os.getcwd()
search_string="MaVI4"
for filename in os.listdir(folder_path):
    if search_string in filename:
        files.append(filename)

In [ ]:
results={}

# Iterating through videos

for video in files:

    # Defining counters

    vehicles_3 = 0
    vehicles_2 = 0

    motorbikes_3=0
    motorbikes_2=0

    cars_2=0
    cars_3=0

    transporters_2=0
    transporters_3=0

    trucks_2=0
    trucks_3=0

    motorbikes_3_list=[]
    motorbikes_2_list=[]

    cars_2_list=[]
    cars_3_list=[]

    transporters_2_list=[]
    transporters_3_list=[]

    trucks_2_list=[]
    trucks_3_list=[]

    # Loading videos
    
    cap = cv2.VideoCapture(video)
    
    cap.set(3, 620)
    cap.set(4, 480)
     
    if cap.isOpened():
       ret, frame1 = cap.read()
    else:
       ret = False
    ret, frame1 = cap.read()
    ret, frame2 = cap.read()
    
    frame_nr=0
    frame_3_counter=0
    frame_2_counter=0

    fps = cap.get(cv2.CAP_PROP_FPS)
    max_frames = int(fps * 30)  # Frames for the first 30 seconds

    # Define video writer to save the output
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Using XVID codec
    out = cv2.VideoWriter('output_first_30_seconds.avi', fourcc, fps, (620, 480))
    
    while ret:
    
       frame_3_counter+=1
       frame_2_counter+=1

       # Frame difference calculations and preprocessing
    
       d = cv2.absdiff(frame1, frame2)
       grey = cv2.cvtColor(d, cv2.COLOR_BGR2GRAY)
     
       blur = cv2.GaussianBlur(grey, (5, 5), 0)
     
       ret, th = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
       dilated = cv2.dilate(th, np.ones((3, 3)))
       kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
     
      
       closing = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, kernel)
       contours, h = cv2.findContours(
           closing, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
       for(i, c) in enumerate(contours):
           (x, y, w, h) = cv2.boundingRect(c)
           contour_valid = (w >= min_contour_width) and (h >= min_contour_height) and (h <= max_contour_height) and (w <= max_contour_width)
     
           if not contour_valid:
               continue

           # Drawing rectangles over centroids
           
           cv2.rectangle(frame1, (x-10, y-10), (x+w+10, y+h+10), (255, 0, 0), 2)

           # Drawing lines
     
           cv2.line(frame1, (lane_3_width_1, lane_3_height_1), (lane_3_width_2, lane_3_height_2), (0, 255, 0), 5)
           cv2.line(frame1, (lane_2_width_1, lane_2_height_1), (lane_2_width_2, lane_2_height_2), (0, 255, 0), 5)
           centrolid = get_centrolid(x, y, w, h)
           matches.append(centrolid)
           cv2.circle(frame1, centrolid, 5, (0, 255, 0), -1)
           cx, cy = get_centrolid(x, y, w, h)

           # Checking if each point in within the given distance from the line, on right width and classifying vehicle type
           
           for (x, y) in matches:
               if (distance_between_points(x, y, new_x_3, new_y_3)>40 or frame_3_counter<20) and x < lane_2_width_1+3 and x > lane_2_width_2-3 and point_to_line_distance((x, y), lane_2_width_1, lane_2_height_1, lane_2_width_2, lane_2_height_2)<offset:
                    if frame_2_counter<=frame_number_threshold and frame_2_counter>12:
                       if last_2=="motorbike":
                           motorbikes_2-=1
                           try:
                               motorbikes_2_list.pop()
                           except:
                               pass
                       if last_2=="car":
                           cars_2-=1
                           try:
                               cars_2_list.pop()
                           except:
                               pass
                       if last_2=="transporter":
                           transporters_2-=1
                           try:
                               transporters_2_list.pop()
                           except:
                               pass
                       if last_2=="truck":
                           trucks_2-=1
                           try:
                               trucks_2_list.pop()
                           except:
                               pass
                       trucks_2+=1
                       trucks_2_list.append(frame_nr)
                       frame_2_counter=0
                    elif frame_2_counter<=12 and frame_2_counter>7:
                       if last_2=="motorbike":
                           motorbikes_2-=1
                           try:
                               motorbikes_2_list.pop()
                           except:
                               pass
                       if last_2=="car":
                           cars_2-=1
                           try:
                               cars_2_list.pop()
                           except:
                               pass
                       if last_2=="transporter":
                           transporters_2-=1
                           try:
                               transporters_2_list.pop()
                           except:
                               pass
                       if last_2=="truck":
                           trucks_2-=1
                           try:
                               trucks_2_list.pop()
                           except:
                               pass
                       transporters_2+=1
                       transporters_2_list.append(frame_nr)
                       frame_2_counter=0
                    elif frame_2_counter>frame_number_threshold:
                       if w<=40 and h<=40:
                           motorbikes_2+=1
                           motorbikes_2_list.append(frame_nr)
                           last_2="motorbike"
                       elif w>40 and h>40 and w<70 and h<70:
                           cars_2+=1
                           cars_2_list.append(frame_nr)
                           last_2="car"
                       elif w<80 and h<80 and w>=70 and h>=70:
                           transporters_2+=1
                           transporters_2_list.append(frame_nr)
                           last_2="transporter"
                       elif w>=75 and h>=75:
                           trucks_2+=1
                           trucks_2_list.append(frame_nr)
                           last_2="truck"
                       else:
                           cars_2+=1
                           cars_2_list.append(frame_nr)
                           last_2="car"
                       frame_2_counter=0
                       vehicles_2+=1
                    try:
                       matches.remove((x, y))
                    except:
                       pass
               if (distance_between_points(x, y, new_x_2, new_y_2)>40 or frame_2_counter>20) and x < lane_3_width_1+2 and x > lane_3_width_2-3 and point_to_line_distance((x, y), lane_3_width_1, lane_3_height_1, lane_3_width_2, lane_3_height_2)<offset:
                    if frame_3_counter<=frame_number_threshold and frame_3_counter>12:
                       if last_3=="motorbike":
                           motorbikes_3-=1
                           try:
                               motorbikes_3_list.pop()
                           except:
                               pass
                       if last_3=="car":
                           cars_3-=1
                           try:
                               cars_3_list.pop()
                           except:
                               pass
                       if last_3=="transporter":
                           transporters_3-=1
                           try:
                               transporters_3_list.pop()
                           except:
                               pass
                       if last_3=="truck":
                           trucks_3-=1
                           try:
                               trucks_3_list.pop()
                           except:
                               pass
                       trucks_3+=1
                       trucks_3_list.append(frame_nr)
                       frame_3_counter=0
                    elif frame_3_counter<=12 and frame_3_counter>7:
                       if last_3=="motorbike":
                           motorbikes_3-=1
                           try:
                               motorbikes_3_list.pop()
                           except:
                               pass
                       if last_3=="car":
                           cars_3-=1
                           try:
                               cars_3_list.pop()
                           except:
                               pass
                       if last_3=="transporter":
                           transporters_3-=1
                           try:
                               transporters_3_list.pop()
                           except:
                               pass
                       if last_3=="truck":
                           trucks_3-=1
                           try:
                               trucks_3_list.pop()
                           except:
                               pass
                       transporters_3+=1
                       transporters_3_list.append(frame_nr)
                       frame_3_counter=0
                    elif frame_3_counter>frame_number_threshold:
                       if w<=40 and h<=40:
                           motorbikes_3+=1
                           motorbikes_3_list.append(frame_nr)
                           last_3="motorbike"
                       elif w>40 and h>40 and w<60 and h<60:
                           cars_3+=1
                           cars_3_list.append(frame_nr)
                           last_3="car"
                       elif w<70 and h<70 and w>=60 and h>=60:
                           transporters_3+=1
                           transporters_3_list.append(frame_nr)
                           last_3="transporter"
                       elif w>=80 and h>=80:
                           trucks_3+=1
                           trucks_3_list.append(frame_nr)
                           last_3="truck"
                       else:
                           cars_3+=1
                           cars_3_list.append(frame_nr)
                           last_3="car"
                       frame_3_counter=0
                       vehicles_3+=1
                    try:
                       matches.remove((x, y))
                    except:
                       pass

       # Text with current counts on the video
     
       cv2.putText(frame1, f"Total Vehicle Detected Lane 3: {vehicles_3}", (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                   (0, 170, 0), 2)
       cv2.putText(frame1, f"Motorbikes: {motorbikes_3}, cars: {cars_3}, transporterds: {transporters_3}, trucks: {trucks_3}", (0, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.4,
                   (170, 0, 0), 1)
       cv2.putText(frame1, f"Total Vehicle Detected Lane 2: {vehicles_2}", (0, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                   (0, 170, 0), 2)
       cv2.putText(frame1, f"Motorbikes: {motorbikes_2}, cars: {cars_2}, transporters: {transporters_2}, trucks: {trucks_2}", (0, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.4,
                   (170, 0, 0), 1)
       cv2.imshow("Vehicle Detection", frame1)

       out.write(frame1)
        
       if cv2.waitKey(1) == 27:
           break
       # Wait for a specified amount of time
       if cv2.waitKey(15) & 0xFF == ord('q'):
            break

       frame1 = frame2
       ret, frame2 = cap.read()
       frame_nr+=1

    cv2.destroyAllWindows()
    cap.release()

    # Saving results into dictionary

    results[video]={"motorbikes_lane_2":motorbikes_2, "motorbikes_lane_2_frames":motorbikes_2_list, "cars_lane_2":cars_2, "cars_lane_2_frames":cars_2_list, "transporters_lane_2":transporters_2, "transporters_lane_2_frames":transporters_2_list, "trucks_lane_2":trucks_2, "trucks_lane_2_frames":trucks_2_list,
    "motorbikes_lane_3":motorbikes_3, "motorbikes_lane_3_frames":motorbikes_3_list, "cars_lane_3":cars_3, "cars_lane_3_frames":cars_3_list, "transporters_lane_3":transporters_3, "transporters_lane_3_frames":transporters_3_list, "trucks_lane_3":trucks_3, "trucks_lane_3_frames":trucks_3_list}

In [ ]:
# Saving into json file

with open(f"{search_string}.json", "w") as json_file:
    json.dump(results, json_file)